In [1]:
import pandas as pd
# Load in data 
train_split = pd.read_pickle("pandas_dumps/compounds_split.pkl")
all_data = pd.read_csv("activity_data.csv")

In [2]:
all_data = all_data.merge(right=train_split,how="right")
# drop any record that has nan for f_avg_
all_data = all_data[all_data['f_avg_IC50'].notna()]
import numpy as np
all_data['pIC50'] = - all_data.f_avg_IC50.apply(np.log10)

In [26]:
all_data.test

0       False
1       False
2       False
3       False
4       False
        ...  
2032    False
2033    False
2034    False
2035    False
2036    False
Name: test, Length: 1901, dtype: bool

# Training on molecular fingerprints


In [51]:
from flaml import AutoML
from sklearn.datasets import fetch_california_housing
X_train = all_data[  ~all_data['test']]['fingerprint']
y_train = all_data[  ~all_data.test]['pIC50']


In [52]:
from rdkit.DataStructs import cDataStructs
X_train = X_train.to_numpy()

In [60]:
X_train = ([np.array(x) for x in X_train])

In [65]:
for item in X_train:
    if item.size != 2048:
        print(item.size)

1024
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
512
512
1024
1024
1024
1024
1024
1024
1024
1024
256
256
256
256
1024
256
1024
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
256
1024
1024
512
1024
1024
512
1024
1024
1024
1024
1024
1024
1024
512
512
1024
1024
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
512
512
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024
1024


In [59]:
X_train

array([array([1, 1, 1, ..., 1, 1, 1]), array([1, 1, 1, ..., 1, 1, 1]),
       array([1, 1, 1, ..., 1, 1, 1]), ...,
       array([1, 0, 1, ..., 0, 1, 1]), array([1, 1, 1, ..., 0, 0, 1]),
       array([1, 1, 1, ..., 0, 0, 1])], dtype=object)

In [ ]:
# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 1,  # in seconds
    "metric": 'r2',
    "task": 'regression',
    "log_file_name": "california.log",
}
X_train, y_train = fetch_california_housing(return_X_y=True)
# Train with labeled input data
automl.fit(X_train=X_train, y_train=y_train,
           **automl_settings)
# Predict
print(automl.predict(X_train))
# Print the best model
print(automl.model.estimator)